In [1]:
import warnings
import igraph as ig
import networkx as nx
import collections
import scipy as sp
import numpy as np
import bMatching as bm
import  Evaluation as ev
import importlib
import matplotlib.pylab as plt
import time
import pandas as pd

In [2]:
def read_graphml(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    
    G1 = ig.read(f1,format="graphml")
    G2 = ig.read(f2,format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [3]:
def read_networks(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    T=nx.read_leda(f1)
    nx.write_graphml(T,'graph.graphml')
    G1 = ig.read('graph.graphml',format="graphml")
    
    T=nx.read_leda(f2)
    nx.write_graphml(T,'graph.graphml')
    G2 = ig.read('graph.graphml',format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [4]:
def get_alignment(G1,G2,M):
    
    n1=G1.vcount()
    n2=G2.vcount()
    Align=set([])
   
    for (i,j) in M:    
        #print(i,j)
        u=G1.vs[i]['id']
        v=G2.vs[j]['id'] 
        #print(u,v)
        Align.add((u,v))
   
    return Align 

In [5]:
def save_alignment(L,filename):
    
    fl=open(filename,"w")
    for (u,v) in L:
        t=u+" "+v+"\n"
        fl.write(t)
    
    fl.close()

In [6]:
def evaluate(AQ,G1,G2,AL):
    
    AQ.mapping_set=AL
    if AQ.true_mapping_set == None:
        qual=AQ.evaluate(False,False,False,False,False,True,False,False,False,False)
        score=qual["NCV-GS3"]
    else:
        qual=AQ.evaluate(False,False,True,False,False,True,False,False,False,False)
        score=float(qual["F-NC"])
    
    return score, qual

In [7]:
def igraph2mtx(G,n,fname='graph.mtx',bipartite=False):
    c1='%%MatrixMarket matrix coordinate real general'
    c2='% Generated'
    
    edgeG=G.get_edgelist()
    #print(edgeG)
    m=int(len(edgeG))
    
    try:
        weights=G.es['weight']
    except:
        weights=[1]*m

    
    if bipartite:
        nl=n
        nr=G.vcount()-nl
        h=str(nl)+" "+str(nr)+" "+str(m)
        
    else:
        nl=n
        nr=n
        h=str(nl)+" "+str(nr)+" "+str(m*2)
    
    f=open(fname,'w')
    
    f.write(c1+"\n")
    f.write(c2+"\n")
    f.write(h+"\n")
    #print(c1)
    #print(c2)
    #print(h)
    for i in range(m):
        (u,v)=edgeG[i]
        w=weights[i]
        
        if bipartite:
            v=v-nl
            #print(u,v,w)
            f.write(str(u+1)+" "+str(v+1)+" "+str(w)+"\n")
        else:
            #print(u,v,w)
            #print(v,u,w)
            f.write(str(u+1)+" "+str(v+1)+" "+str(w)+"\n")
            f.write(str(v+1)+" "+str(u+1)+" "+str(w)+"\n")
    
    f.close()

In [8]:
def netAlignSC(f1,f2,fl,alpha=1,beta=1,ft=None,input_dir=None):

    G1,G2,L=read_networks(f1,f2,fl,input_dir)

    igraph2mtx(G1,G1.vcount(),fname='graph-A.mtx')
    igraph2mtx(G2,G2.vcount(),fname='graph-B.mtx')
    igraph2mtx(L,G1.vcount(),fname='graph-L.mtx',bipartite=True)


    #### Initialize evaluation class
    if ft is not None:
        AQ=ev.AlignmentQuality(input_dir+f1, input_dir+f2, None, input_dir+ft, None, None)
    else:
        AQ=ev.AlignmentQuality(input_dir+f1, input_dir+f2, None, None, None, None)

    #!../netalignmc/netalign/netAlign graph -n 40 -a $alpha -b $beta -o sc.aln
    os.system('/home/khan242/netAlign/netalignmc/netalign/netAlign graph -n 40 -v')

    f=open('sc.aln','r')
    M=[]
    for l in f:
        l=l.strip('\n')
        l=l.split(' ')
        M.append((int(l[0])-1,int(l[1])-1))
    f.close()
    AL=get_alignment(G1,G2,M)
    s,q=evaluate(AQ,G1,G2,AL)
    #!rm sc.aln graph-*  
    
    return AL,s,q

In [ ]:
input_dir="/home/khan242/netAlign/data/synthetic_networks/"

f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"
fl="L_0.25_yeast0_yeast5.graphml"
ft="true_node_mapping.txt"

G1,G2,L=read_networks(f1,f2,fl,input_dir)

igraph2mtx(G1,G1.vcount(),fname='graph-A.mtx')
igraph2mtx(G2,G2.vcount(),fname='graph-B.mtx')
igraph2mtx(L,G1.vcount(),fname='graph-L.mtx',bipartite=True)
# AL,s,q=netAlignSC(f1,f2,fl,alpha,beta,ft,input_dir=input_dir)

# for alpha in [0,0.25,0.50,0.75,1.00]:
    
#     beta=1-alpha
#     AL,s,q=netAlignSC(f1,f2,fl,alpha,beta,ft,input_dir=input_dir)
#     save_alignment(AL,res_dir+resf)
#     print(s,q)    



In [ ]:
for i in range(res.shape[0]):
    for j in range(res.shape[1]):
        res.iloc[i][j]=round(res.iloc[i][j],3)
res
pd.to_pickle(res,"results_t1.pkl")

In [16]:
res=pd.read_pickle("results_v1.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.357,0.369,0.386,0.367,0.351
a=0.75,0.397,0.484,0.272,0.465,0.394
a=0.5,0.399,0.426,0.410,0.325,0.385
a=0.25,0.399,0.492,0.303,0.505,0.370
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [17]:
res=pd.read_pickle("results_synth.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.378,0.310,0.385,0.401,0.334
a=0.75,0.438,0.501,0.372,0.409,0.308
a=0.5,0.331,0.376,0.304,0.372,0.507
a=0.25,0.332,0.333,0.133,0.343,0.366
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [18]:
res=pd.read_pickle("results_t1.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.435,0.419,0.426,0.440,0.434
a=0.75,0.417,0.420,0.417,0.448,0.471
a=0.5,0.412,0.470,0.443,0.457,0.442
a=0.25,0.487,0.459,0.372,0.331,0.519
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [19]:
res=pd.read_pickle("results_t2.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.480,0.485,0.491,0.477,0.491
a=0.75,0.504,0.506,0.504,0.508,0.486
a=0.5,0.527,0.565,0.523,0.645,0.105
a=0.25,0.589,0.590,0.530,0.675,0.014
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [20]:
res=pd.read_pickle("results_t3.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.172,0.176,0.177,0.160,0.173
a=0.75,0.174,0.211,0.220,0.194,0.182
a=0.5,0.194,0.252,0.187,0.235,0.271
a=0.25,0.195,0.207,0.204,0.230,0.274
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [21]:
res=pd.read_pickle("results_t4.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.172,0.174,0.171,0.171,0.175
a=0.75,0.195,0.208,0.236,0.219,0.189
a=0.5,0.207,0.282,0.247,0.232,0.295
a=0.25,0.209,0.305,0.257,0.246,0.205
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [22]:
res=pd.read_pickle("results_danai.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.664,0.714,0.707,0.725,0.675
a=0.75,0.757,0.801,0.747,0.732,0.764
a=0.5,0.748,0.799,0.718,0.715,0.737
a=0.25,0.129,0.075,0.726,0.704,0.675
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [2]:
res=pd.read_pickle("results_gwave.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.045,0.053,0.051,0.057,0.044
a=0.75,0.336,0.532,0.476,0.381,0.070
a=0.5,0.393,0.419,0.294,0.164,0.024
a=0.25,0.308,0.424,0.244,0.127,0.046
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [2]:
res=pd.read_pickle("results_Ginx.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.248,0.287,0.301,0.273,0.273
a=0.75,0.481,0.514,0.310,0.212,0.225
a=0.5,0.371,0.455,0.284,0.206,0.151
a=0.25,0.429,0.356,0.242,0.186,0.215
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [3]:
res=pd.read_pickle("results_Ginx1.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.248,0.282,0.298,0.273,0.273
a=0.75,0.481,0.436,0.356,0.238,0.225
a=0.5,0.474,0.473,0.284,0.214,0.158
a=0.25,0.417,0.330,0.257,0.189,0.215
a=0,-1.000,-1.000,-1.000,-1.000,0.581


In [3]:
res=pd.read_pickle("results_scone.pkl")
res

,K=5%,K=10%,K=25%,K=50%,K=100%
a=1,0.394,0.376,0.382,0.368,0.239
a=0.75,0.371,0.443,0.385,0.385,0.380
a=0.5,0.357,0.378,0.416,0.378,0.422
a=0.25,0.367,0.461,0.253,0.305,0.489
a=0,-1.000,-1.000,-1.000,-1.000,0.581
